# 어텐션의 구조

- seq2seq를 한층 더 강력하게 하는 **어텐션 매커니즘**
- 인간처럼 필요한 정보에만 "주목"할 수 있게 만든다
- seq2seq의 근본적인 문제를 해결한다

## seq2seq의 문제점

- Encoder의 출력은 "고정 길이의 벡터"
    - 고정 길이 벡터는 입력 문장의 길이에 관계없이 항상 같은 길이의 벡터로 변환하게 해준다
    - **아무리 긴 벡터여도 고정 길이에 우겨넣어짐**

<img src = "../imgs/fig 8-1.png" width="400">

## Encoder 개선

- Encoder 출력의 길이는 입력 문장에 길이에 따라 바꿔주는 것이 좋다! (당연)
- 구체적으로는, **시각별 LSTM 계층의 은닉 상태 벡터를 모두 이용하는 것**

<img src = "../imgs/fig 8-2.png" width="500">


#### <center>===> 이것으로 Encoder는 "하나의 고정 길이 벡터"라는 제약에서 벗어남</center>

- (참고) RNN 계층을 초기화할 때 두 가지 반환 중 선택 가능
    - 모든 시각의 은닉 상태 벡터 반환 
        - [keras] `return_sequences=True`
    - 마지막 은닉 상태 벡터만 반환
    
    
- 각 시각의 은닉 상태에는 직전에 입력된 단어에 대한 정보가 많이 포함되어 있다.
    - 예를 들어 `나` `는` `고양이` `로소` `이다` 에서 `고양이` 벡터에는 `나` `는` `고양이` 의 정보가 담겨 있다
    - 주변 정보를 균형있게 담기 위해서는 **양방향 RNN** 이 효과적임

## Decoder 개선 ①

<img src = "../imgs/fig 8-5.png" width="700">

- 앞 장의 Decoder는 Encoder의 LSTM 계층의 마지막 은닉 상태만을 이용한다 
- hs에서 마지막 줄만 빼내어 Decoder에 전달한 것

#### ===> hs 전부를 활용할 수 있도록 개선

---

- 사람이 문장 `나는 고양이로소이다`를 영어로 번역한다면? 
    - 나 = I, 고양이 = cat
    - 나와 고양이가 문장의 주를 이루므로 나와 고양이라는 단어에 "주목" 하면서 번역을 하게 됨. 나 = I, 고양이 = cat 와 같은 단어 간 대응 관계(`alignment`)를 미리 알고 있다면 번역 효과가 더 좋을 것임.
    - **대응 관계 (`alignment`) 아이디어 를 활용하는 것이 바로 어텐션 매커니즘**
    
---

#### 입력과 출력의 단어 간 대응 관계를 seq2seq에게 학습시키자!!!

- `도착어 단어`와 `대응 관계`에 있는 `출발어 단어`의 정보를 골라내면 더 학습(번역)이 잘 될 것 이다!
- **필요한 정보에만 주목하여 그 정보로부터 시계열 변환을 수행하는 것이 목표**



---
<img src = "../imgs/fig 8-6.png" width="700">

- `어떤 계산`이 받는 입력 두 가지
    1. Encoder로부터 받는 hs
    2. 시각 별 LSTM 계층의 은닉 상태
       - 여기서 필요한 정보만 골라 위쪽의 Affine계층으로 출력한다
- Decoder의 첫번째 계층에 마지막 은닝 상태 벡터 hs를 전달하는 것은 유지

- 목적은 **단어들의 대응관계 추출**
- 각 시각에서 Decoder에 입력된 단어와 대응관계인 단어의 벡터를 hs에서 골라내자.
- 예를 들어 Decoder가 `I`를 출력할 때 hs에서 `나`에 대응하는 벡터를 선택함.
    - 그러나 선택하는 작업은 미분할 수 없다. ( = 오차역전파 안됨 )
    - 선택이 아니라 " " 

## Decoder 개선 ②

## Decoder 개선 ③